In [2]:
import argparse
import os
import torch
import gc
import pandas as pd
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.chains import create_retrieval_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader




/home/ubuntu/anaconda3/envs/anlp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2", model_kwargs={"device": "cuda"})
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={"device": "cuda"})
vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)

In [ ]:
def batch_inference(qa_chain, questions, batch_size, ground_truths=None):
    """
    Processes the list of questions in batches and returns accumulated predictions.
    If ground_truths are provided, computes and prints metrics for each batch.
    """
    predictions = []
    num_batches = (len(questions) + batch_size - 1) // batch_size

    for i in range(0, len(questions), batch_size):
        batch = questions[i:i+batch_size]
        batch_inputs = [{"query": q} for q in batch]
        # Use invoke to process the batch
        with torch.no_grad():
            if batch_size > 1:
                batch_outputs = qa_chain.batch(batch_inputs)
            else:
                batch_outputs = [qa_chain.invoke(batch_inputs[0])]
        torch.cuda.empty_cache()
        gc.collect()
        
        batch_predictions = []
        for output in batch_outputs:
            # Handle different output formats (dict or string)
            if isinstance(output, dict):
                answer = output.get("result", output)
            else:
                answer = output
            batch_predictions.append(answer)
        
        predictions.extend(batch_predictions)
        
        # If ground truths are provided, compute and print batch metrics
        if ground_truths:
            batch_ground_truths = ground_truths[i:i+batch_size]
            batch_metrics = calculate_metrics(batch_predictions, batch_ground_truths)
            print(f"Batch {i//batch_size + 1}/{num_batches} Metrics:")
            for key, value in batch_metrics.items():
                print(f"  {key}: {value}")
            print("-" * 40)
            
    return predictions
def remove_punctuation(sentence):
    if sentence[-1] == ".":
        return sentence[:-1]
    else:
        return sentence

def calculate_metrics(predictions, ground_truths):
    """
    Calculates evaluation metrics (recall, F1, exact match) for a list of predictions
    compared to ground truths.
    """
    # Ensure the lengths match
    assert len(predictions) == len(ground_truths), "Lengths of predictions and ground truths must match."
    total_examples = len(predictions)
    
    exact_match_count = 0
    total_recall = 0.0
    total_precision = 0.0

    for pred, gt in zip(predictions, ground_truths):
        pred = pred.lower()
        gt = gt.lower()
        # Count exact matches
        if pred == gt:
            exact_match_count += 1
        
        # Tokenize the prediction and ground truth
        pred_tokens = set(pred.split())
        gt_tokens = set(gt.split())
        
        # Compute intersection of tokens
        common_tokens = pred_tokens & gt_tokens
        
        # Compute recall and precision for the current pair
        recall = len(common_tokens) / len(gt_tokens) if gt_tokens else 0
        precision = len(common_tokens) / len(pred_tokens) if pred_tokens else 0
        
        total_recall += recall
        total_precision += precision

    # Compute average recall and precision
    avg_recall = total_recall / total_examples
    avg_precision = total_precision / total_examples
    
    # Calculate F1 score (harmonic mean)
    f1_score = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall) if (avg_precision + avg_recall) > 0 else 0
    exact_match = exact_match_count / total_examples

    return {
        "recall": avg_recall,
        "f1": f1_score,
        "exact_match": exact_match,
        "exact_match_count": exact_match_count,
        "total_examples": total_examples,
        "total_recall": total_recall,
        "total_precision": total_precision
    }
def extract_answer(answer):
    import re

    # Use a regex pattern to capture everything after "Helpful Answer:" until the end of the text
    match = re.search(r"(?i)Helpful Answer:\s*(.*)", answer, re.DOTALL)
    if match:
        answer_part = match.group(1).strip()
        answer_part = answer_part.split("\n")[0]
        return answer_part
    else:
        return "Extract failed"

In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device {device}")

Using device cuda


In [6]:
topk = 3
# generation_model_name = "google/flan-t5-base"
generation_model_name = "Qwen/Qwen2-7B-Instruct"
ground_truths = None

retriever = vectorstore.as_retriever(search_kwargs={"k": topk})
tokenizer = AutoTokenizer.from_pretrained(generation_model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(generation_model_name, device_map="auto")
model = AutoModelForCausalLM.from_pretrained(generation_model_name, torch_dtype=torch.float16, device_map="auto")
# model.config.use_cache = False
model.eval()
# pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=512)
pipe = pipeline('text-generation', model=model, tokenizer=tokenizer, torch_dtype=torch.float16)
llm = HuggingFacePipeline(pipeline=pipe)
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Loading checkpoint shards: 100%|██████████| 4/4 [00:07<00:00,  1.77s/it]
Device set to use cuda:0


In [7]:
question = "What department handles emergency medical services in the city of Pittsburgh?"
relevant_docs = vectorstore.similarity_search(question, k=3)
for doc in relevant_docs:
    print(f"file: {doc.metadata}")
for doc in relevant_docs:
    print(f"content: {doc.page_content}")
    

file: {'source': '/home/ubuntu/ThreeRiversRAG/data/retrieve_source/total_web_txt/425.txt'}
file: {'source': '/home/ubuntu/ThreeRiversRAG/data/retrieve_source/total_web_txt/445.txt'}
file: {'source': '/home/ubuntu/ThreeRiversRAG/data/retrieve_source/total_web_txt/425.txt'}
content: is overseen by a minimum of two district chiefs during normal daily operations. medic units are spread across pittsburgh better allowing for the most effective and efficient response. also, in coordination with the center for emergency medicine, a physician is able to respond to the scene at the request of our paramedics to assist with the treatment of particularly critical patients. in addition to emergency treatment and transport of patients, the ambulance division provides medical coverage
content: pittsburgh through the worst of the crisis, helped to protect employees, residents, and visitors,” added director schmidt. pittsburgh is home to a unique emergency medical services (ems) system where physicians 

In [12]:
# Define your custom instruction prompt
ans_req = """Answer the question based only on the provided context in just one sentence. Each answer must be extremely succinct—limited to just several keywords—and should not repeat the question."""
yes_or_no_req = """If a question is a yes or no question, the answer must be exactly 'yes' or 'no' without any additional information."""
instruction_prompt = " ".join([ans_req, yes_or_no_req])
# Combine the instruction prompt with the question.
final_prompt = f"{instruction_prompt}\nQuestion: {question}"



predictions = batch_inference(qa_chain, [final_prompt], 1, ground_truths if ground_truths else None)
print(extract_answer(predictions[0]))

{'query': "Answer the question based only on the provided context in just one sentence. Each answer must be extremely succinct—limited to just several keywords—and should not repeat the question. If a question is a yes or no question, the answer must be exactly 'yes' or 'no' without any additional information.\nQuestion: What department handles emergency medical services in the city of Pittsburgh?", 'result': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\napplications & forms one stop pgh permits licenses bulletins community resources code enforcement condemned buildings pli fees building codes appeals pli contacts announcements registrations innovation & performance sub-menu pittsburgh digital equity coalition public works sub-menu about public works contact public works environmental services architecture division street maintenance bureau forestry division zoning s

In [83]:
ans = ["yes"]
gt = ["Yes, there's the Climate Action Plan in place."]
calculate_metrics(ans, gt)

yes
yes, there's the climate action plan in place


{'recall': 0.0,
 'f1': 0,
 'exact_match': 0.0,
 'exact_match_count': 0,
 'total_examples': 1,
 'total_recall': 0.0,
 'total_precision': 0.0}